In [4]:
# works like charm
# converts .pdf files in selected directory to .txt files

from tika import parser
import os
import re
from nltk import word_tokenize
from nltk.corpus import stopwords

# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file 

def extract_text_from_pdfs_recursively(dir):
    for root, dirs, files in os.walk(dir):
        for file in files:
            path_to_pdf = os.path.join(root, file)
            [stem, ext] = os.path.splitext(path_to_pdf)
            #s_words = set(stopwords.words('english'))
            #print(s_words)
            if ext == '.pdf':
                #print("Processing " + path_to_pdf)
                pdf_contents = parser.from_file(path_to_pdf)
                
                # problem: saves the txts to the same directory
                
                path_to_txt = stem + '.txt'
                with open(path_to_txt, 'w') as txt_file:
                    #print("Writing contents to " + path_to_txt)
                    
                    # a bit ugly...
                    
                    pdf_contents = str(pdf_contents['content'].encode('utf-8', errors='ignore')).replace("\n", "").replace("\\", "").lower()
                    pdf_contents = re.sub("(\\d|\\W)+"," ",pdf_contents)
                    
                    
                    #pdf_contents = word_tokenize(pdf_contents)
                    #pdf_contents = [x for x in pdf_contents if x not in s_words]
                    
                    txt_file.write(str(pdf_contents))
                    


if __name__ == "__main__":
    extract_text_from_pdfs_recursively("c:/Users/soirk/Krisztian/Egyetem/Survey Statisztika Msc/Szakdolgozat/pdfs/")

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/soirk/Krisztian/Egyetem/Survey Statisztika Msc/Szakdolgozat/pdfs/10.1136_bmjopen-2018-028767/txts/.txt'

In [90]:
# searching for keywords
# ~working

# https://stackoverflow.com/questions/47649987/how-to-save-nltk-concordance-results-in-a-list

def concordance(ci, word, width=75, lines=100):
    """
    Rewrite of nltk.text.ConcordanceIndex.print_concordance that returns results
    instead of printing them. 

    See:
    http://www.nltk.org/api/nltk.html#nltk.text.ConcordanceIndex.print_concordance
    """
    half_width = (width - len(word) - 2) // 2
    context = width // 4 # approx number of words of context

    results = []
    offsets = ci.offsets(word)
    if offsets:
        lines = min(lines, len(offsets))
        for i in offsets:
            if lines <= 0:
                break
            left = (' ' * half_width +
                    ' '.join(ci._tokens[i-context:i]))
            right = ' '.join(ci._tokens[i+1:i+context])
            left = left[-half_width:]
            right = right[:half_width]
            results.append('%s %s %s' % (left, ci._tokens[i], right))
            lines -= 1

    return results

# https://stackoverflow.com/questions/29110950/python-concordance-command-in-nltk
# http://www.nltk.org/book/ch03.html

import nltk.corpus  
from nltk import word_tokenize
from nltk.text import Text  
from nltk import ConcordanceIndex

path = 'c:/Users/soirk/Krisztian/Egyetem/Survey Statisztika Msc/Szakdolgozat/txts/'

# searching for 'missing' in one paper

paper = open(path+'Briggs-2003-Missing-presumed-at-random-cost-ana.txt','rb')

paper_token = word_tokenize(str(paper.read()))

paper_txt = Text(paper_token)



# occurence of 'missing' in the text

ci = ConcordanceIndex(paper_txt.tokens)

paper_conc = str(concordance(ci,'missing'))

paper_conc

#len(paper_txt.concordance_list('missing')) # gives only the length of the displayed results

#len(ConcordanceIndex(paper_txt.tokens).offsets('missing')) # gives the correct number of results


7900

In [91]:
# making dataframe 
# works fine

import os
import numpy as np
import pandas as pd
import nltk.corpus  
from nltk import word_tokenize
from nltk.text import Text  
from nltk import ConcordanceIndex


path = "c:/Users/soirk/Krisztian/Egyetem/Survey Statisztika Msc/Szakdolgozat/txts/"
filelist = os.listdir(path)
filelist_sort = filelist[0:5]
sentences_list = []
papers_list = []
keywords = ["missing","impute","imputing","imputation","imputed","pairwise","listwise","deletion","delete","deleted"]

papers = pd.DataFrame(filelist)
sentences_df = pd.DataFrame(filelist)
values_list = []

for j in range(len(keywords)):
    for i in range(len(filelist)):
    #print(filelist[i])
        paper = open(path+filelist[i],'rb')

        paper_token = word_tokenize(str(paper.read()))

        paper_txt = Text(paper_token)
        
        ci = ConcordanceIndex(paper_txt.tokens)

        sentences = str(concordance(ci,keywords[j]))
    
        value = len(ConcordanceIndex(paper_txt.tokens).offsets(keywords[j]))
        #print(keywords[j],':',value)
        
        og_dict = {keywords[j]:value}
        values_list.append(value)
        sentences_list.append(sentences)
    
    papers = pd.concat([papers.reset_index(drop=True), pd.DataFrame(values_list, columns = [keywords[j]])], axis=1)
    sentences_df = pd.concat([sentences_df.reset_index(drop=True), pd.DataFrame(sentences_list, columns = [keywords[j]])], axis=1)
    
    
    values_list = []
    sentences_list = []

#papers.columns = ['paper',"missing","impute","imputing","imputation","imputed","pairwise","listwise","deletion","delete","deleted"]

papers


,0,missing,impute,imputing,imputation,imputed,pairwise,listwise,deletion,delete,deleted
0,10.1136_bmjopen-2018-028767.txt,3,0,0,0,0,0,0,0,0,0
1,10.1371_journal.pcbi.1007399.txt,88,12,2,37,9,0,0,0,0,0
2,Banbura-2014-Maximum-likelihood-estimation-of-...,78,0,0,0,0,0,0,0,0,0
3,Briggs-2003-Missing-presumed-at-random-cost-an...,134,2,6,61,19,0,1,3,0,0
4,Bunji-Okada2019_Article_ItemResponseAndRespons...,1,0,0,0,0,0,0,0,0,0
5,clinical stuff - tresiba and ryzodeg evaluatio...,20,3,0,7,6,0,0,0,0,0
6,Deng-2016-Multiple-imputation-for-general-mis.txt,60,7,4,62,15,0,0,0,0,0
7,Donald B. Rubin - Multiple Imputation for Nonr...,284,40,10,465,142,1,0,0,0,0
8,dudbrigde-likelihood-based association analysi...,40,0,0,2,0,0,0,0,0,0
9,"Gareth James, Daniela Witten, Trevor Hastie, R...",14,0,0,0,0,8,0,0,1,0


In [92]:
papers.to_csv(path +'papers.csv')
sentences_df.to_csv(path +'sentences.csv')

In [43]:
og_dict

og_dict_list

[{'missing': 3},
 {'missing': 88},
 {'missing': 78},
 {'missing': 134},
 {'missing': 1},
 {'impute': 0},
 {'impute': 12},
 {'impute': 0},
 {'impute': 2},
 {'impute': 0},
 {'imputing': 0},
 {'imputing': 2},
 {'imputing': 0},
 {'imputing': 6},
 {'imputing': 0},
 {'imputation': 0},
 {'imputation': 37},
 {'imputation': 0},
 {'imputation': 61},
 {'imputation': 0},
 {'imputed': 0},
 {'imputed': 9},
 {'imputed': 0},
 {'imputed': 19},
 {'imputed': 0},
 {'pairwise': 0},
 {'pairwise': 0},
 {'pairwise': 0},
 {'pairwise': 0},
 {'pairwise': 0},
 {'listwise': 0},
 {'listwise': 0},
 {'listwise': 0},
 {'listwise': 1},
 {'listwise': 0},
 {'deletion': 0},
 {'deletion': 0},
 {'deletion': 0},
 {'deletion': 3},
 {'deletion': 0},
 {'delete': 0},
 {'delete': 0},
 {'delete': 0},
 {'delete': 0},
 {'delete': 0},
 {'deleted': 0},
 {'deleted': 0},
 {'deleted': 0},
 {'deleted': 0},
 {'deleted': 0},
 {'missing': 3},
 {'missing': 88},
 {'missing': 78},
 {'missing': 134},
 {'missing': 1},
 {'impute': 0},
 {'impute': 

In [ ]:
# dictionary

class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 
        
dict_obj = my_dictionary() 
  
dict_obj.add(1, 'Geeks') 
dict_obj.add(2, 'forGeeks') 
  
print(dict_obj) 
        
#

list